In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
import pylatexenc


def compute_response(content):
    n = content['n']
    m = content['m']
    gates = content['gates']
    input_states = content['cats']
    output = content['output']
    
    circuit = Circuit(n, m, gates, input_states, output)
    return circuit.run()
    

def place_quantum_gate(gate, qc, row):
    
    if gate == 'x':
        qc.x(row)
    elif gate == 'y':
        qc.y(row)
    elif gate == 'z':
        qc.z(row)
    elif gate == 'h':
        qc.h(row)
    elif gate == 'i':
        qc.i(row)


def place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc):
    if gate == 'x':
        qc.cx(controlled_qubit, conditional_qubit)
    elif gate == 'y':
        qc.cy((controlled_qubit, conditional_qubit))
    elif gate == 'z':
        qc.cz((controlled_qubit, conditional_qubit))
    elif gate == 'h':
        qc.ch((controlled_qubit, conditional_qubit))
    elif gate == 'i':
        qc.ci((controlled_qubit, conditional_qubit))

        
def prepare_input_state(qc, qr, input_string):
    for i, bit in enumerate(input_string):
        if bit == '1':
            qc.x(qr[i])
    return qc
        

def handle_results(qc, qr, cr):
    qc.measure(qr, cr)

    simulator = Aer.get_backend('qasm_simulator')
    job = execute(qc, simulator, shots=10)

    result = job.result()
    counts = result.get_counts(qc)

    print(counts)

    print(qc)
    qc.draw('mpl')
    
    return json.dumps(counts)
        

class Circuit:
    def __init__(self, n, m, gates_matrix, input_states, output_states):
        self.n = n
        self.m = m
        self.gates_matrix = gates_matrix
        self.input_states = input_states
        self.output_states = output_states

    def run(self):
        qr = QuantumRegister(self.n)
        cr = ClassicalRegister(self.n)
        qc = QuantumCircuit(qr, cr)
        
        prepare_input_state(qc, qr, self.input_states)

        for row in range(self.n):
            for column in range(self.m):
                gate_info = self.gates_matrix[row][column]
                
                if gate_info:
                
                    is_controlled, info = gate_info

                    if is_controlled:
                        controlled_qubit, conditional_qubit, gate = info
                        place_controlled_quantum_gate(controlled_qubit, conditional_qubit, gate, qc)
                    else:
                        place_quantum_gate(info, qc, row)

        return handle_results(qc, qr, cr)


In [ ]:
# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
import time

import json

hostName = "localhost"
serverPort = 8082

class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes(json.dumps(a, ensure_ascii=False), 'utf-8'))
    
    def do_POST(self):
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        
        content = json.loads(post_body)
        content_dict = json.loads(content)
        print('Request with body: ', content_dict)
        
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()

        message = compute_response(content_dict)
        self.wfile.write(bytes(message, "utf8"))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:8082
Request with body:  {'n': 1, 'm': 1, 'gates': [[[False, 'h']]], 'cats': '0', 'output': [1]}


127.0.0.1 - - [10/Jun/2023 14:54:00] "POST / HTTP/1.1" 200 -


{'0': 7, '1': 3}
      ┌───┐┌─┐
  q0: ┤ H ├┤M├
      └───┘└╥┘
c0: 1/══════╩═
            0 
